In [ ]:
# upgrade mlxtend

!pip install mlxtend --upgrade --no-deps

In [ ]:
from google.colab import drive
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.models import load_model
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import cv2
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten,GlobalAveragePooling2D,BatchNormalization,Input
import random
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau,TensorBoard,LearningRateScheduler,EarlyStopping,ModelCheckpoint
import time
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import Model
import os

In [ ]:
NAME = "MobileNetV2 2DL"
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/logs/{}'.format(NAME))

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
IMG_HEIGHT = IMG_WIDTH = 100
BATCH_SIZE = 32
CHANNEL_NUMBER = 3
EPOCHS = 50
LEARNING_RATE = 1e-4
DROPOUT = 0.20
SEED = 19

In [ ]:
# mount drive

drive.mount('/content/drive')

In [ ]:
# show directory

!ls "/content/drive/My Drive/Licenta/Dataset"

In [ ]:
# copy the .zip file into the root directory of cloud VM

!cp "/content/drive/My Drive/Licenta/DatasetApple.zip" ../

In [ ]:
# unzip the zip file

!unzip ../DatasetApple.zip -d ../

In [ ]:
# set paths
test_ds = '/FourFruitsTest'
test_ds_crop = '/FourFruitsTestCrop'
test_path = '/DatasetApple/Test'
train_path = '/DatasetApple/Train'

In [ ]:
# image transformation

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   vertical_flip=True,
                                   horizontal_flip=True,
                                   validation_split=0.10, 
                                   fill_mode='nearest'  
                                   )


test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# take images from directory

train_image_generator = train_datagen.flow_from_directory(train_path,
                                                            target_size=(IMG_HEIGHT,IMG_WIDTH),
                                                            color_mode='rgb',
                                                            batch_size=BATCH_SIZE,
                                                            shuffle=True,
                                                            subset='training',
                                                            class_mode='categorical',
                                                            seed=42)

validation_image_generator = train_datagen.flow_from_directory(train_path,
                                                            target_size=(IMG_HEIGHT,IMG_WIDTH),
                                                            color_mode='rgb',
                                                            subset='validation',
                                                            batch_size= BATCH_SIZE,
                                                            class_mode='categorical',
                                                            shuffle=False,
                                                            seed=42
                                                            )

test_image_generator = test_datagen.flow_from_directory(test_path,
                                                            target_size=(IMG_HEIGHT,IMG_WIDTH),
                                                            color_mode='rgb',
                                                            batch_size= BATCH_SIZE,
                                                            class_mode='categorical',
                                                            shuffle=False,
                                                            seed=42)

In [ ]:
# list with names of classes

class_names = []

dictionar = train_image_generator.class_indices
for key in dictionar.keys():
  class_names.append(key)
class_names

In [ ]:
# XCEPTION LEARNING TRANSFER

base_model = Xception(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights="imagenet", include_top=False)
base_model.trainable = False

inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model(inputs, training=False)
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(13, activation='softmax')(x)
model = Model(inputs,outputs)
model.summary()

In [ ]:
# MobileNetV2 LEARNING TRANSFER

base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights="imagenet", include_top=False)
base_model.trainable = False

inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model(inputs, training=False)
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(13, activation='softmax')(x)
model = Model(inputs,outputs)
model.summary()

In [ ]:
# VGG16 LEARNING TRANSFER

base_model = VGG16(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights="imagenet", include_top=False)
base_model.trainable = False

inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model(inputs, training=False)
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)

outputs = Dense(13, activation='softmax')(x)
model = Model(inputs,outputs)
model.summary()

In [ ]:
# CNN 

model = Sequential()
model.add(Conv2D(filters=16,kernel_size=(3,3),input_shape=(IMG_HEIGHT,IMG_WIDTH,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(IMG_HEIGHT,IMG_WIDTH,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(IMG_HEIGHT,IMG_WIDTH,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),input_shape=(IMG_HEIGHT,IMG_WIDTH,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))


model.add(Flatten())
# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(13,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       7

In [ ]:
 learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, 
                                                factor=0.5, min_lr=0.00001)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

results = model.fit(train_image_generator,
                    epochs=50,
                    steps_per_epoch=(train_image_generator.samples // BATCH_SIZE) ,
                    validation_data=validation_image_generator,
                    validation_steps=(validation_image_generator.samples // BATCH_SIZE) ,
                    callbacks=[tensorboard,learning_rate_reduction])

In [ ]:
model.save('MobileNetV2_2DL.h5')

In [ ]:
early = EarlyStopping(patience=3)

In [ ]:
base_model.trainable = True

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

results = model.fit(train_image_generator,
                    epochs=75,
                    initial_epoch=50,
                    steps_per_epoch=(train_image_generator.samples // BATCH_SIZE) ,
                    validation_data=validation_image_generator,
                    validation_steps=(validation_image_generator.samples // BATCH_SIZE) ,
                    callbacks=[tensorboard,early])

In [ ]:
model.save('MobileNetV2_2DL_FT.h5')

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/logs

In [ ]:
Model = load_model('/content/drive/MyDrive/Model/CNN.h5')
Model_1DL = load_model('/content/drive/MyDrive/Model/CNN_1DL.h5')
Model_2DL = load_model('/content/drive/MyDrive/Model/CNN_2DL.h5')

In [ ]:
Model.evaluate(test_image_generator,verbose=1)
Model_1DL.evaluate(test_image_generator,verbose=1)
Model_2DL.evaluate(test_image_generator,verbose=1)

In [ ]:
Model = load_model('/content/drive/MyDrive/Model/MobileNetV2.h5')
Model_1DL = load_model('/content/drive/MyDrive/Model/MobileNetV2_1DL.h5')
Model_2DL = load_model('/content/drive/MyDrive/Model/MobileNetV2_2DL.h5')
Model_FT = load_model('/content/drive/MyDrive/Model/VGG16_FT.h5')
Model_1DL_FT = load_model('/content/drive/MyDrive/Model/MobileNetV2_1DL_FT.h5')
Model_2DL_FT = load_model('/content/drive/MyDrive/Model/MobileNetV2_2DL_FT.h5')

In [ ]:
Model.evaluate(test_image_generator,verbose=1)
Model_1DL.evaluate(test_image_generator,verbose=1)
Model_2DL.evaluate(test_image_generator,verbose=1)
Model_FT.evaluate(test_image_generator,verbose=1)
Model_1DL_FT.evaluate(test_image_generator,verbose=1)
Model_2DL_FT.evaluate(test_image_generator,verbose=1)

In [ ]:
testare ='/Test/'
lista = os.listdir('/Test/')


path_list = []

plt.figure(figsize=(13, 30))
i = 0
for item in lista:
  path = testare+item
  path_list.append(path)

  myimage = image.load_img(path,target_size=(IMG_WIDTH,IMG_HEIGHT))
  my_img_array = image.img_to_array(myimage) / 255
  my_img_array_show = image.img_to_array(myimage)/255
  myimage_pred = np.expand_dims(my_img_array,axis=0)


  prediction =Model.predict(myimage_pred)
  pred_sort =(-prediction).argsort()
  prediction_prob = sorted(prediction[0],reverse=True)

  names = []
  for item in pred_sort[0]:
    names.append(class_names[item])
  
  print(names)
  # print(pred_sort[0])
  print(prediction_prob)
  print(class_names[pred_sort[0][0]]+ '\n')


  
  i = i + 1
  ax = plt.subplot(8, 4, i)
  plt.imshow(my_img_array_show)
  plt.title(class_names[pred_sort[0][0]]+" | " +str(np.round(prediction_prob[0]*100,3)) + " %",fontdict={'color': 'white'})
  plt.axis("off")
  
  pred_list = []
  for item in pred_sort[0]:
    pred_list.append(class_names[item])
  pred_list = reversed(pred_list)

  

In [ ]:
testare ='/Tes/'
lista = os.listdir('/Tes/')


path_list = []

plt.figure(figsize=(20, 20))
i = 0
for item in lista:
  path = testare+item
  path_list.append(path)

  myimage = image.load_img(path,target_size=(IMG_WIDTH,IMG_HEIGHT))

  my_img_array_show = image.img_to_array(myimage)/255



  
  i = i + 1
  ax = plt.subplot(7, 8, i)
  plt.imshow(my_img_array_show)

  plt.axis("off")


In [ ]:
!rm -rf '/Test'

In [ ]:
!zip -r "/content/logs.zip" "/content/logs"

In [ ]:
Modele.evaluate(test_image_generator_ds_crop,verbose=1)

In [ ]:
Modele = load_model('/content/CNN.h5')

In [ ]:
pred = Model_1DL.predict(test_image_generator)

In [ ]:
predictions = np.argmax(pred, axis=1)

In [ ]:
# display confusion matrix

cm = confusion_matrix(test_image_generator.classes,predictions)

fig, ax = plot_confusion_matrix(conf_mat=cm,
                                colorbar=True,
                                show_absolute=False,
                                show_normed=True,
                                figsize=(10,10),
                                class_names = class_names)